# Generate random observation data for ZTF broad-band (gri)
- Gregory S.H. Paek
- 22.10.05

In [1]:
import glob, os
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack
import matplotlib as mpl
from astropy import units as u
import speclite.filters
import time
import multiprocessing
from itertools import repeat

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 200
plt.rc('font', family='serif')

## Useful functions

In [3]:
from helper import makeSpecColors
from helper import convert_flam2fnu
from helper import get_wollaeger
from helper import convert_app2abs
from helper import convert_abs2app
kncbtbl = get_wollaeger()
from helper import get_bandwidth_table
bdwtbl = get_bandwidth_table()
#	speclite
from helper import get_speclite_med
meds = get_speclite_med()
mlam = meds.effective_wavelengths
mbdw = bdwtbl['bandwidth'][bdwtbl['group']=='Med']*u.Angstrom
from helper import get_speclite_sdss
sdss = get_speclite_sdss()
slam = sdss.effective_wavelengths
sbdw = bdwtbl['bandwidth'][bdwtbl['group']=='SDSS']*u.Angstrom
from helper import get_speclite_jc
jc = get_speclite_jc()
jclam = jc.effective_wavelengths
jcbdw = bdwtbl['bandwidth'][bdwtbl['group']=='Johnson Cousin']*u.Angstrom
from helper import get_speclite_lsst
from helper import get_lsst_bandwidth
lsstbdw = get_lsst_bandwidth()
lsst = get_speclite_lsst()
lsstlam = lsst.effective_wavelengths
from helper import get_ztf_depth

# lsstbdw = bdwtbl['bandwidth'][bdwtbl['group']
#
from helper import get_7dt_depth
from helper import get_7dt_broadband_depth
# dptbl = get_7dt_depth(exptime=180)
dptbl = get_7dt_broadband_depth(exptime=180)

In [4]:
magoffset = -4.3

In [5]:
knsp = '../3.table/kn_sim_cube_synphot/Run_TS_dyn_all_lanth_wind2_all_md0.1_vd0.3_mw0.1_vw0.3_angle0_synphot_lsst.ecsv'
intbl = ascii.read(knsp)
intbl[10:25]

t,u,g,r,i,z,y
d,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),mag(AB)
float64,float64,float64,float64,float64,float64,float64
0.297,-14.058,-13.748,-13.390,-13.173,-12.890,-12.622
0.324,-14.038,-13.776,-13.433,-13.215,-12.963,-12.728
0.354,-13.981,-13.786,-13.466,-13.262,-13.022,-12.812
0.386,-13.875,-13.776,-13.494,-13.292,-13.075,-12.894
0.42,-13.699,-13.747,-13.530,-13.318,-13.123,-12.990
0.459,-13.430,-13.685,-13.568,-13.350,-13.183,-13.065
0.5,-13.067,-13.578,-13.609,-13.393,-13.245,-13.129
0.545,-12.635,-13.411,-13.646,-13.456,-13.307,-13.199


- Synphot Table

In [6]:
def extract_param_kn_sim_cube(knsp):
	part = os.path.basename(knsp).split('_')

	if part[1] == 'TP':
		dshape = 'toroidal'
	elif part[1] == 'TS':
		dshape = 'spherical'
	else:
		dshape = ''

	#	Latitude
	if part[5] == 'wind1':
		lat = 'Axial'
	elif part[5] == 'wind2':
		lat = 'Edge'
	else:
		lat = ''

	#	Ejecta mass for low-Ye [solar mass]
	md = float(part[7].replace('md', ''))

	#	Ejecta velocity for low-Ye [N*c]
	vd = float(part[8].replace('vd', ''))

	#	Ejecta mass for high-Ye [solar mass]
	mw = float(part[9].replace('mw', ''))

	#	Ejecta velocity for high-Ye [N*c]
	vw = float(part[10].replace('vw', ''))

	#	Angle
	angle = float(part[11].replace('angle', ''))

	return dshape, lat, md, vd, mw, vw, angle

In [7]:
def calc_snr(m, ul, sigma=5):
	snr = sigma*10**((ul-m)/5)
	return snr

def convert_snr2magerr(snr):
	merr = 2.5*np.log10(1+1/snr)
	return merr

def calc_GaussianFraction(seeing, optfactor=0.6731, path_plot=None):
	import numpy as np
	import matplotlib.pyplot as plt
	from scipy.special import erf

	# seeing, optfactor= 1.5, 0.6731

	mu = 0.0
	# sigma = fwhm_seeing/2.355
	fwhm2sigma = seeing*2.355
	# optfactor = 0.6731
	sigma = fwhm2sigma*optfactor

	x = np.linspace(-8, 8, 1000)
	y = (1 / np.sqrt(2 * np.pi * sigma**2)) * np.exp(-(x-mu)**2 / (2 * sigma**2))
	y_cum = 0.5 * (1 + erf((x - mu)/(np.sqrt(2 * sigma**2))))

	indx_aperture = np.where(
		(x>-sigma*optfactor) &
		(x<+sigma*optfactor)
	)
	xaper = x[indx_aperture]
	yaper = y[indx_aperture]

	frac = np.sum(yaper)/np.sum(y) 
	# print(np.sum(y), np.sum(yaper), frac)

	if path_plot != None:
		plt.plot(x, y, alpha=0.5, label=f'PDF of N(0, {sigma:1.3f})', lw=5)
		plt.plot(xaper, yaper, alpha=1.0, label=f'Aperture ({frac*1e2:.1f}%)', lw=5,)
		plt.xlabel('x', fontsize=20)
		plt.ylabel('f(x)', fontsize=20)
		plt.legend(loc='lower center', fontsize=14)
		# plt.show()
		plt.savefig(path_plot, overwrite=True)
	else:
		pass

	return frac

def add_noise(mu, sigma, nseed, n=10, path_plot=None):
	"""
	mu, sigma = 17.5, 0.1
	n = 10
	"""
	from scipy.stats import norm
	import numpy as np
	
	try:
		x = np.arange(mu-sigma*n, mu+sigma*n, sigma*1e-3)
		y = norm(mu, sigma).pdf(x)

		if path_plot != None:
			resultlist = []
			for i in range(10000):
				xobs = np.random.choice(x, p=y/np.sum(y))
				# print(xobs)
				resultlist.append(xobs)
			plt.axvspan(xmin=mu-sigma, xmax=mu+sigma, alpha=0.5, color='tomato',)
			plt.axvline(x=mu, ls='--', alpha=1.0, color='tomato', lw=3)
			plt.plot(x, y, lw=3, alpha=0.75, color='grey')
			plt.hist(resultlist, lw=3, alpha=0.75, color='k', histtype='step', density=True)
			plt.xlabel(r'$\rm m_{obs}$')
			plt.plot(x, y)
			plt.savefig(path_plot, overwrite=True)
		else:
			pass
		#	more complicated choice with the fixed random seed
		np.random.seed(int((nseed+1)+(mu*1e2)))
		return np.random.choice(x, p=y/np.sum(y))
	except:
		# print('Something goes wrong (add_noise function)')
		return None

# Iteration

- Setting

In [8]:
path_result = "../5.result/kn_sim_cube_obs"
if ~os.path.exists(path_result): os.system(f"mkdir {path_result}")

aperfrac = calc_GaussianFraction(seeing=1.5, optfactor=0.6731, path_plot=None)
niteration = int(1e0)
ncores = 3

tag = 'decam'
exptime = 90
filterliststr = 'iz'

mkdir: ../5.result/kn_sim_cube_obs: File exists


In [9]:
knsplist = sorted(glob.glob("../3.table/kn_sim_cube_synphot/Run_*_synphot_lsst.ecsv"))
knsplist[:5]

['../3.table/kn_sim_cube_synphot/Run_TP_dyn_all_lanth_wind1_all_md0.001_vd0.05_mw0.001_vw0.05_angle0_synphot_lsst.ecsv',
 '../3.table/kn_sim_cube_synphot/Run_TP_dyn_all_lanth_wind1_all_md0.001_vd0.05_mw0.001_vw0.05_angle120_synphot_lsst.ecsv',
 '../3.table/kn_sim_cube_synphot/Run_TP_dyn_all_lanth_wind1_all_md0.001_vd0.05_mw0.001_vw0.05_angle150_synphot_lsst.ecsv',
 '../3.table/kn_sim_cube_synphot/Run_TP_dyn_all_lanth_wind1_all_md0.001_vd0.05_mw0.001_vw0.05_angle180_synphot_lsst.ecsv',
 '../3.table/kn_sim_cube_synphot/Run_TP_dyn_all_lanth_wind1_all_md0.001_vd0.05_mw0.001_vw0.05_angle30_synphot_lsst.ecsv']

In [10]:
import test

niteration = int(1e0)

t0 = time.time()
ncores = 3
# knsplist0 = knsplist[:5]

# d = 40*u.Mpc

for d in np.arange(40, 200+40, 40):
    d = d*u.Mpc
    print(f"{d}")
    if __name__ == '__main__':
        pool = multiprocessing.Pool(processes=ncores)  # 현재 시스템에서 사용 할 프로세스 개수
        pool.starmap(test.run_table_routine4decam, zip(knsplist, repeat(d), repeat(niteration), repeat(aperfrac), repeat(magoffset), repeat(lsstlam), repeat(lsstbdw)))

        pool.close()
        pool.join()

    folder = f"{path_result}/{tag}_{filterliststr}_{exptime}s_iter{niteration}_{int(d.value)}Mpc"
    os.system(f"mkdir {folder}")
    mvcom = f"mv {path_result}/Run*.ecsv {folder}"
    os.system(mvcom)
    delt = time.time() - t0
    print(f'Done (laps : {delt/60:1.1f} min)')

delt = time.time() - t0
print(f"Time : {delt:1.3f} sec")


40.0 Mpc
Done (laps : 4.2 min)
80.0 Mpc
Done (laps : 8.0 min)
120.0 Mpc
Done (laps : 12.0 min)
160.0 Mpc
Done (laps : 15.9 min)
200.0 Mpc
Done (laps : 19.6 min)
Time : 1175.078 sec
